In [ ]:
import subprocess
import sys
import os
import argparse

def install_package(package):
    """Install package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully!")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        return False

def check_and_install_dependencies():
    """Check and install required dependencies"""
    dependencies_available = True

    try:
        import qrcode
        from PIL import Image
        print("Dependencies are already available!")
        return True
    except ImportError:
        print("Installing dependencies...")

        # Install qrcode with PIL support
        if not install_package("qrcode[pil]"):
            dependencies_available = False

        # Install pillow separately as backup
        if not install_package("pillow"):
            dependencies_available = False

        if dependencies_available:
            try:
                import qrcode
                from PIL import Image
                print("Dependencies installed and imported successfully!")
                return True
            except ImportError as e:
                print(f"Error importing after installation: {e}")
                print("Try running manually: pip install qrcode[pil] pillow")
                return False
        else:
            print("Failed to install required dependencies")
            return False

def validate_url(url):
    """Basic URL validation"""
    if not url:
        return False

    # Add protocol if missing
    if not url.startswith(('http://', 'https://', 'ftp://', 'mailto:')):
        url = 'https://' + url

    return url

def generate_qr_code(url, filename="qr_code.png", size=10, border=4,
                    error_correction='L', fill_color="black", back_color="white",
                    verbose=True):
    """
    Generate QR code from the given URL/link

    Parameters:
    url (str): The URL to convert to QR code
    filename (str): Output file name (default: qr_code.png)
    size (int): Box size of QR code (default: 10)
    border (int): Border size of QR code (default: 4)
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'L')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        import qrcode

        # Validate URL
        validated_url = validate_url(url)
        if not validated_url:
            raise ValueError("Invalid URL provided")

        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }

        if error_correction.upper() not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Validate parameters
        if size < 1 or size > 40:
            raise ValueError("Size must be between 1 and 40")
        if border < 0:
            raise ValueError("Border must be non-negative")

        # Create QR code instance
        qr = qrcode.QRCode(
            version=None,  # Auto-detect version
            error_correction=error_levels[error_correction.upper()],
            box_size=size,
            border=border,
        )

        # Add data (URL) to QR code
        qr.add_data(validated_url)
        qr.make(fit=True)

        # Create QR code image
        img = qr.make_image(fill_color=fill_color, back_color=back_color)

        # Ensure output directory exists
        output_dir = os.path.dirname(filename) if os.path.dirname(filename) else '.'
        os.makedirs(output_dir, exist_ok=True)

        # Save image
        img.save(filename)

        if verbose:
            print(f"QR Code generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {validated_url}")
            print(f"Image size: {img.size}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR code: {str(e)}")
        return False

def generate_qr_with_logo(url, logo_path, filename="qr_code_with_logo.png",
                         size=10, border=4, error_correction='H',
                         fill_color="black", back_color="white",
                         logo_scale=0.2, verbose=True):
    """
    Generate QR code with a logo in the center

    Parameters:
    url (str): The URL to convert to QR code
    logo_path (str): Path to the logo file
    filename (str): Output file name
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'H')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    logo_scale (float): Scale of logo relative to QR size (default: 0.2)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        import qrcode
        from PIL import Image

        # Validate inputs
        if not os.path.exists(logo_path):
            raise FileNotFoundError(f"Logo file not found: {logo_path}")

        validated_url = validate_url(url)
        if not validated_url:
            raise ValueError("Invalid URL provided")

        if not (0.1 <= logo_scale <= 0.4):
            raise ValueError("Logo scale should be between 0.1 and 0.4 for optimal readability")

        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }

        if error_correction.upper() not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Create base QR code
        qr = qrcode.QRCode(
            version=None,
            error_correction=error_levels[error_correction.upper()],
            box_size=size,
            border=border,
        )

        qr.add_data(validated_url)
        qr.make(fit=True)

        # Create QR code image
        qr_img = qr.make_image(fill_color=fill_color, back_color=back_color).convert("RGBA")

        # Open and process logo
        try:
            logo = Image.open(logo_path).convert("RGBA")
        except Exception as e:
            raise ValueError(f"Unable to open logo file: {e}")

        # Calculate logo size
        qr_width, qr_height = qr_img.size
        logo_size = int(min(qr_width, qr_height) * logo_scale)

        # Resize logo with high quality resampling
        logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

        # Calculate center position for logo
        logo_pos = ((qr_width - logo_size) // 2, (qr_height - logo_size) // 2)

        # Create a white background for the logo area (optional, for better visibility)
        logo_bg = Image.new('RGBA', (logo_size + 20, logo_size + 20), (255, 255, 255, 255))
        logo_bg_pos = ((qr_width - logo_size - 20) // 2, (qr_height - logo_size - 20) // 2)
        qr_img.paste(logo_bg, logo_bg_pos, logo_bg)

        # Paste logo onto QR code
        qr_img.paste(logo, logo_pos, logo)

        # Ensure output directory exists
        output_dir = os.path.dirname(filename) if os.path.dirname(filename) else '.'
        os.makedirs(output_dir, exist_ok=True)

        # Save image
        qr_img.save(filename, quality=95)

        if verbose:
            print(f"QR Code with logo generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {validated_url}")
            print(f"Logo: {logo_path}")
            print(f"Final image size: {qr_img.size}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR code with logo: {str(e)}")
        return False

def batch_generate_qr(urls_list, prefix="qr_", size=10, border=4,
                     error_correction='L', fill_color="black", back_color="white",
                     verbose=True):
    """
    Generate multiple QR codes from a list of URLs

    Parameters:
    urls_list (list): List of URLs
    prefix (str): Prefix for file names
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level
    fill_color (str): Fill color
    back_color (str): Background color
    verbose (bool): Print status messages

    Returns:
    int: Number of QR codes successfully created
    """
    if not urls_list:
        if verbose:
            print("No URLs provided for batch generation")
        return 0

    success_count = 0
    failed_urls = []

    for i, url in enumerate(urls_list, 1):
        filename = f"{prefix}{i:03d}.png"  # Use zero-padded numbering

        if verbose:
            print(f"Processing URL {i}/{len(urls_list)}: {url[:50]}...")

        if generate_qr_code(url, filename, size, border, error_correction,
                           fill_color, back_color, verbose=False):
            success_count += 1
            if verbose:
                print(f"✓ Generated: {filename}")
        else:
            failed_urls.append((i, url))
            if verbose:
                print(f"✗ Failed: {url}")

        if verbose:
            print("-" * 50)

    if verbose:
        print(f"\nBatch generation completed!")
        print(f"Successfully generated: {success_count}/{len(urls_list)} QR codes")
        if failed_urls:
            print(f"Failed URLs:")
            for idx, url in failed_urls:
                print(f"  {idx}: {url}")

    return success_count

def interactive_mode():
    """Interactive mode for user input"""
    print("\n=== QR Code Generator - Interactive Mode ===")
    print("1: Generate single QR code")
    print("2: Generate batch QR codes from file")
    print("3: Generate QR code with logo")

    while True:
        try:
            mode = input("\nChoose mode (1-3): ").strip()
            if mode in ['1', '2', '3']:
                break
            print("Please enter 1, 2, or 3")
        except KeyboardInterrupt:
            print("\nExiting...")
            sys.exit(0)

    if mode == "1":
        return handle_single_qr_interactive()
    elif mode == "2":
        return handle_batch_qr_interactive()
    elif mode == "3":
        return handle_logo_qr_interactive()

def handle_single_qr_interactive():
    """Handle single QR code generation in interactive mode"""
    url = input("Enter URL: ").strip()
    if not url:
        print("URL is required.")
        return False

    filename = input("Enter output filename (default: qr_code.png): ").strip() or "qr_code.png"

    # Add .png extension if not present
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        filename += '.png'

    size = get_int_input("Enter box size (1-40, default: 10): ", 10, 1, 40)
    border = get_int_input("Enter border size (0-10, default: 4): ", 4, 0, 10)

    error_correction = input("Enter error correction level (L/M/Q/H, default: L): ").strip().upper() or 'L'
    if error_correction not in ['L', 'M', 'Q', 'H']:
        error_correction = 'L'

    fill_color = input("Enter fill color (default: black): ").strip() or "black"
    back_color = input("Enter background color (default: white): ").strip() or "white"

    return generate_qr_code(url, filename, size, border, error_correction, fill_color, back_color)

def handle_batch_qr_interactive():
    """Handle batch QR code generation in interactive mode"""
    urls_file = input("Enter path to URLs file: ").strip()
    if not urls_file or not os.path.exists(urls_file):
        print("Valid URLs file path is required.")
        return False

    prefix = input("Enter prefix for batch files (default: qr_): ").strip() or "qr_"
    size = get_int_input("Enter box size (1-40, default: 10): ", 10, 1, 40)
    border = get_int_input("Enter border size (0-10, default: 4): ", 4, 0, 10)

    error_correction = input("Enter error correction level (L/M/Q/H, default: L): ").strip().upper() or 'L'
    if error_correction not in ['L', 'M', 'Q', 'H']:
        error_correction = 'L'

    fill_color = input("Enter fill color (default: black): ").strip() or "black"
    back_color = input("Enter background color (default: white): ").strip() or "white"

    try:
        with open(urls_file, 'r', encoding='utf-8') as f:
            urls = [line.strip() for line in f if line.strip() and not line.startswith('#')]

        if not urls:
            print("No valid URLs found in file.")
            return False

        return batch_generate_qr(urls, prefix, size, border, error_correction, fill_color, back_color)
    except Exception as e:
        print(f"Error reading file: {e}")
        return False

def handle_logo_qr_interactive():
    """Handle QR code with logo generation in interactive mode"""
    url = input("Enter URL: ").strip()
    if not url:
        print("URL is required.")
        return False

    logo_path = input("Enter logo file path: ").strip()
    if not logo_path or not os.path.exists(logo_path):
        print("Valid logo file path is required.")
        return False

    filename = input("Enter output filename (default: qr_code_with_logo.png): ").strip() or "qr_code_with_logo.png"

    # Add .png extension if not present
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        filename += '.png'

    size = get_int_input("Enter box size (1-40, default: 10): ", 10, 1, 40)
    border = get_int_input("Enter border size (0-10, default: 4): ", 4, 0, 10)

    error_correction = input("Enter error correction level (L/M/Q/H, default: H): ").strip().upper() or 'H'
    if error_correction not in ['L', 'M', 'Q', 'H']:
        error_correction = 'H'

    fill_color = input("Enter fill color (default: black): ").strip() or "black"
    back_color = input("Enter background color (default: white): ").strip() or "white"

    logo_scale = get_float_input("Enter logo scale (0.1-0.4, default: 0.2): ", 0.2, 0.1, 0.4)

    return generate_qr_with_logo(url, logo_path, filename, size, border, error_correction,
                               fill_color, back_color, logo_scale)

def get_int_input(prompt, default, min_val=None, max_val=None):
    """Get integer input with validation"""
    while True:
        try:
            value_str = input(prompt).strip()
            if not value_str:
                return default
            value = int(value_str)
            if min_val is not None and value < min_val:
                print(f"Value must be at least {min_val}")
                continue
            if max_val is not None and value > max_val:
                print(f"Value must be at most {max_val}")
                continue
            return value
        except ValueError:
            print("Please enter a valid integer")
        except KeyboardInterrupt:
            print("\nExiting...")
            sys.exit(0)

def get_float_input(prompt, default, min_val=None, max_val=None):
    """Get float input with validation"""
    while True:
        try:
            value_str = input(prompt).strip()
            if not value_str:
                return default
            value = float(value_str)
            if min_val is not None and value < min_val:
                print(f"Value must be at least {min_val}")
                continue
            if max_val is not None and value > max_val:
                print(f"Value must be at most {max_val}")
                continue
            return value
        except ValueError:
            print("Please enter a valid number")
        except KeyboardInterrupt:
            print("\nExiting...")
            sys.exit(0)

def main():
    """Main function"""
    # Check dependencies first
    if not check_and_install_dependencies():
        print("Cannot proceed without required dependencies")
        sys.exit(1)

    parser = argparse.ArgumentParser(
        description="Advanced QR Code Generator",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  python qr_generator.py --url https://example.com
  python qr_generator.py --url https://example.com --logo logo.png --output custom_qr.png
  python qr_generator.py --urls-file urls.txt --prefix batch_qr_
  python qr_generator.py  # Interactive mode
        """
    )

    parser.add_argument("--url", type=str, help="URL to generate QR code for")
    parser.add_argument("--urls-file", type=str, help="File containing list of URLs (one per line) for batch generation")
    parser.add_argument("--logo", type=str, help="Path to logo file for embedding")
    parser.add_argument("--output", type=str, default="qr_code.png", help="Output file name (default: qr_code.png)")
    parser.add_argument("--size", type=int, default=10, help="Box size 1-40 (default: 10)")
    parser.add_argument("--border", type=int, default=4, help="Border size 0-10 (default: 4)")
    parser.add_argument("--error-correction", type=str, default='L',
                       choices=['L', 'M', 'Q', 'H', 'l', 'm', 'q', 'h'],
                       help="Error correction level (default: L)")
    parser.add_argument("--fill-color", type=str, default="black", help="Fill color (default: black)")
    parser.add_argument("--back-color", type=str, default="white", help="Background color (default: white)")
    parser.add_argument("--logo-scale", type=float, default=0.2, help="Logo scale 0.1-0.4 (default: 0.2)")
    parser.add_argument("--prefix", type=str, default="qr_", help="Prefix for batch file names (default: qr_)")
    parser.add_argument("--quiet", action="store_true", help="Suppress status messages")

    # Handle Jupyter notebook arguments
    args, unknown = parser.parse_known_args()

    # Filter out Jupyter-specific arguments
    jupyter_args = ['-f', '--ip', '--port', '--transport', '--heartbeat']
    unknown = [arg for arg in unknown if not any(arg.startswith(j) for j in jupyter_args)]

    verbose = not args.quiet

    # If no arguments provided, enter interactive mode
    if len(sys.argv) == 1 or (not args.url and not args.urls_file):
        if verbose:
            print("No arguments provided. Entering interactive mode.")
        try:
            interactive_mode()
        except KeyboardInterrupt:
            print("\nExiting...")
        return

    # Validate arguments
    if args.size < 1 or args.size > 40:
        print("Error: Size must be between 1 and 40")
        sys.exit(1)

    if args.border < 0 or args.border > 10:
        print("Error: Border must be between 0 and 10")
        sys.exit(1)

    if args.logo_scale < 0.1 or args.logo_scale > 0.4:
        print("Error: Logo scale must be between 0.1 and 0.4")
        sys.exit(1)

    # Process based on input type
    try:
        if args.urls_file:
            # Batch generation
            if not os.path.exists(args.urls_file):
                print(f"Error: URLs file not found: {args.urls_file}")
                sys.exit(1)

            with open(args.urls_file, 'r', encoding='utf-8') as f:
                urls = [line.strip() for line in f if line.strip() and not line.startswith('#')]

            if not urls:
                print("Error: No valid URLs found in file")
                sys.exit(1)

            success_count = batch_generate_qr(urls, args.prefix, args.size, args.border,
                                            args.error_correction.upper(), args.fill_color,
                                            args.back_color, verbose)

            if verbose:
                print(f"\nBatch processing completed: {success_count}/{len(urls)} successful")

        elif args.url:
            # Single QR generation
            if args.logo:
                if not os.path.exists(args.logo):
                    print(f"Error: Logo file not found: {args.logo}")
                    sys.exit(1)

                success = generate_qr_with_logo(args.url, args.logo, args.output, args.size,
                                              args.border, args.error_correction.upper(),
                                              args.fill_color, args.back_color,
                                              args.logo_scale, verbose)
            else:
                success = generate_qr_code(args.url, args.output, args.size, args.border,
                                         args.error_correction.upper(), args.fill_color,
                                         args.back_color, verbose)

            if not success:
                sys.exit(1)
        else:
            print("Error: No valid input provided")
            parser.print_help()
            sys.exit(1)

    except KeyboardInterrupt:
        print("\nOperation cancelled by user")
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Dependencies are already available!
No arguments provided. Entering interactive mode.

=== QR Code Generator - Interactive Mode ===
1: Generate single QR code
2: Generate batch QR codes from file
3: Generate QR code with logo

Choose mode (1-3): 1
Enter URL: https://outlook.office.com/mail/inbox/id/AAQkADFmOWJjZmI3LThlNTItNDQzMS1iYWMwLTU0YmNjZDcxOGUxMgAQAGrIYl1QNSVChnKviSOOixs%3D
Enter output filename (default: qr_code.png): 
Enter box size (1-40, default: 10): 20\
Please enter a valid integer
Enter box size (1-40, default: 10): 20
Enter border size (0-10, default: 4): 5
Enter error correction level (L/M/Q/H, default: L): h
